In [1]:
!nvidia-smi

Sat Aug  1 14:12:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers

     |████████████████████████████████| 778kB 2.9MB/s 
     |████████████████████████████████| 3.0MB 10.3MB/s 
     |████████████████████████████████| 1.1MB 38.6MB/s 
     |████████████████████████████████| 890kB 46.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=439db0daf91233c038ba9385534c922c31a929bede038c8175ec8e2589fcaa8b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 2.8MB/s 


In [4]:
!pip install ujson

     |████████████████████████████████| 184kB 2.8MB/s 


In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
!unzip -q "/content/gdrive/My Drive/Yelp_Sentiment_Analysis/dataset.zip" -d '/content'

**Fetch data and Preprocess**

In [7]:
import random
import numpy as np
import json
from collections import Counter
from tqdm import tqdm
import re
from transformers import BertTokenizer, BertModel

In [8]:
# does data cleaning for inital text
def process_file(filename, data_type):
    print(f"Pre-processing {data_type} examples...")
    examples = {}
    total = 0

    with open(filename, "r") as fh:
        source = json.load(fh)
        total = len(source)

    for idx in tqdm(range(total)):
        text = source[str(idx)]["text"]
        # basic text preprocessing
        text = text.replace("''", '" ').replace("``", '" ')  # replace the quotes 
        text = text.replace("`", "'") # backticks typo
        text = text.replace("\"", "") # replace quotes
        text = text.replace("...", " ").replace(". . .", " ").replace('..', ' ') # replace dots
        text = text.replace("\n", " ") # replace new line chars
        text = re.sub(r'(?:http:|https:).*?(?=\s)', '', text)  # remove url and website
        text = re.sub(r'www.*?(?=\s)', '', text)  # remove url and website

        list_to_replace = [':(', '=)', ':)', ':P', '-', ',,', ':', ';', '/', '+', '~', '_', '*', '(', ')', '&', '='] #replace the punctuations which are messy with empty
        for elem in list_to_replace:
            text = text.replace(elem, '')
        
        text = re.sub(r'\!{2,}', '!', text) # duplicate punctuatio
        text = re.sub(r'\?{2,}', '!', text) # duplicate punctuation
        text = text.replace('?!', '?').replace('!?', '?') #replace slang punctuation with question
        text = re.sub(r'\s(?:\.|\,)', '', text) # replace spaces before punctuation
        text = re.sub(r'([a-zA-Z?!])\1\1+', r'\1', text) # removes repeated characters (Ex: Veryyyyy -> very)

        text = re.sub(r'\s{2,}', ' ', text) # replace multiple spaces
        text = text.strip() # strips spaces
        text = text.lower() # lower text

        examples[str(idx)] = {"user_id": source[str(idx)]["user_id"], 
                              "business_id": source[str(idx)]["business_id"], 
                              "text": text,
                              "rating": source[str(idx)]["rating"]}

    return examples

In [9]:
train_examples = process_file('dataset/train.json', "train")
test_examples = process_file('dataset/test.json', "test")
test_examples_small = process_file('dataset/test_small.json', "test")

Pre-processing train examples...


  7%|▋         | 741/10000 [00:00<00:01, 7408.84it/s]

Pre-processing test examples...


100%|██████████| 100/100 [00:00<00:00, 6852.21it/s]

Pre-processing test examples...


In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
# Key Tokens
print('[CLS] token: ', tokenizer.convert_tokens_to_ids("[CLS]"))
print('[SEP] token: ', tokenizer.convert_tokens_to_ids("[SEP]"))
print('[PAD] token: ', tokenizer.convert_tokens_to_ids("[PAD]"))

[CLS] token:  101
[SEP] token:  102
[PAD] token:  0


In [12]:
def pad_sent(dataset, max_text_len = 350):  #token number 0 is [PAD]
    text = []
    attnmask = []
    seg_id = []
    rating = []
    uniq_ids = []

    for idx in tqdm(dataset):
        uniq_ids.append(int(idx))  # append unique ids
        curr_text = dataset[str(idx)]["text"] # extract text
        curr_text = "[CLS] " + curr_text  # add starting cls token
        tokenized_text = tokenizer.tokenize(curr_text) # tokenize
        tokenized_ids = tokenizer.convert_tokens_to_ids(tokenized_text) # convert to ids
        
        tokenized_ids = tokenized_ids[:max_text_len - 1]   # trim the reviews
        tokenized_ids.append(102) # add special token for [SEP]

        curr_sent_len = len(tokenized_ids)
        remaining = max_text_len - curr_sent_len # words remaining for padding

        # pad the input token
        tokenized_ids.extend([0] * remaining)  # pad the text to max_text_len
        # add to storage
        text.append(tokenized_ids)
        
        # create attention and segmented mask
        curr_attn = [1] * curr_sent_len  
        curr_attn.extend([0] * remaining)
        curr_seg_id = [0] * max_text_len

        # append to storage
        attnmask.append(curr_attn)
        seg_id.append(curr_seg_id)
        rating.append(dataset[str(idx)]["rating"])

    return text, attnmask, seg_id, rating, uniq_ids

In [13]:
text, attnmask, seg_id, rating, uniq_ids = pad_sent(train_examples)

100%|██████████| 90000/90000 [03:50<00:00, 391.01it/s]


In [14]:
np.savez('/content/gdrive/My Drive/Yelp_Sentiment_Analysis/train.npz',
         text = np.array(text),
         attnmask = np.array(attnmask),
         seg_id = np.array(seg_id),
         rating = np.array(rating),
         ids = np.array(uniq_ids)
)

In [15]:
text, attnmask, seg_id, rating, uniq_ids = pad_sent(test_examples)

100%|██████████| 10000/10000 [00:27<00:00, 360.44it/s]


In [16]:
np.savez('/content/gdrive/My Drive/Yelp_Sentiment_Analysis/test.npz',
         text = np.array(text),
         attnmask = np.array(attnmask),
         seg_id = np.array(seg_id),
         rating = np.array(rating),
         ids = np.array(uniq_ids)
)

In [17]:
text, attnmask, seg_id, rating, uniq_ids = pad_sent(test_examples_small)

100%|██████████| 100/100 [00:00<00:00, 345.24it/s]


In [18]:
np.savez('/content/gdrive/My Drive/Yelp_Sentiment_Analysis/test_small.npz',
         text = np.array(text),
         attnmask = np.array(attnmask),
         seg_id = np.array(seg_id),
         rating = np.array(rating),
         ids = np.array(uniq_ids)
)

**Data Loader**

In [19]:
import torch
import torch.utils.data as data
import numpy as np

In [20]:
class YelpDataset(data.Dataset):
    """Yelp Dataset.

    Each item in the dataset is a tuple with the following entries (in order):
         text = np.array(text),
         attnmask = np.array(attnmask),
         seg_id = np.array(seg_id),
         rating = np.array(rating),
         ids = np.array(uniq_ids)

    Args:
        data_path (str): Path to .npz file containing pre-processed dataset.
    """
    def __init__(self, data_path):
        super(YelpDataset, self).__init__()

        dataset = np.load(data_path)
        self.text = torch.from_numpy(dataset['text']).long()
        self.attnmask = torch.from_numpy(dataset['attnmask']).long()
        self.seg_id = torch.from_numpy(dataset['seg_id']).long()
        self.rating = torch.from_numpy(dataset['rating']).long()
        self.ids = torch.from_numpy(dataset['ids']).long()

        # index
        self.valid_idxs = [idx for idx in range(len(self.ids))]

    def __getitem__(self, idx):
        idx = self.valid_idxs[idx]
        example = (self.text[idx],
                   self.attnmask[idx],
                   self.seg_id[idx],
                   self.rating[idx],
                   self.ids[idx])
        return example

    def __len__(self):
        return len(self.valid_idxs)

In [21]:
train_dataset = YelpDataset('gdrive/My Drive/Yelp_Sentiment_Analysis/train.npz')
train_loader = data.DataLoader(train_dataset,
                                batch_size=8,
                                shuffle=True,
                                num_workers=4,
                                )

dev_dataset = YelpDataset('gdrive/My Drive/Yelp_Sentiment_Analysis/test.npz')
dev_loader = data.DataLoader(dev_dataset,
                                batch_size=8,
                                shuffle=False,
                                num_workers=4,
                                )

**BERT Finetune**

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
#import torch.optim.lr_scheduler as sched
from torch.optim.lr_scheduler import StepLR

from transformers import BertTokenizer, BertModel, BertConfig


import numpy as np
import tqdm
from ujson import load as json_load
from collections import OrderedDict
from json import dumps
import random
import os
import logging
import queue
import shutil
import string
import re

random.seed(224)
np.random.seed(224)
torch.manual_seed(224)
torch.cuda.manual_seed_all(224)

from tensorboardX import SummaryWriter

In [23]:
class AverageMeter:
    """Keep track of average values over time.

    Adapted from:
        > https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    def __init__(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        """Reset meter."""
        self.__init__()

    def update(self, val, num_samples=1):
        """Update meter with new value `val`, the average of `num` samples.

        Args:
            val (float): Average value to update the meter with.
            num_samples (int): Number of samples that were averaged to
                produce `val`.
        """
        self.count += num_samples
        self.sum += val * num_samples
        self.avg = self.sum / self.count


class EMA:
    """Exponential moving average of model parameters.
    Args:
        model (torch.nn.Module): Model with parameters whose EMA will be kept.
        decay (float): Decay rate for exponential moving average.
    """
    def __init__(self, model, decay):
        self.decay = decay
        self.shadow = {}
        self.original = {}

        # Register model parameters
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def __call__(self, model, num_updates):
        decay = min(self.decay, (1.0 + num_updates) / (10.0 + num_updates))
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = \
                    (1.0 - decay) * param.data + decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def assign(self, model):
        """Assign exponential moving average of parameter values to the
        respective parameters.
        Args:
            model (torch.nn.Module): Model to assign parameter values.
        """
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.original[name] = param.data.clone()
                param.data = self.shadow[name]

    def resume(self, model):
        """Restore original parameters to a model. That is, put back
        the values that were in each parameter at the last call to `assign`.
        Args:
            model (torch.nn.Module): Model to assign parameter values.
        """
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                param.data = self.original[name]


class CheckpointSaver:
    """Class to save and load model checkpoints.

    Save the best checkpoints as measured by a metric value passed into the
    `save` method. Overwrite checkpoints with better checkpoints once
    `max_checkpoints` have been saved.

    Args:
        save_dir (str): Directory to save checkpoints.
        max_checkpoints (int): Maximum number of checkpoints to keep before
            overwriting old ones.
        metric_name (str): Name of metric used to determine best model.
        maximize_metric (bool): If true, best checkpoint is that which maximizes
            the metric value passed in via `save`. Otherwise, best checkpoint
            minimizes the metric.
        log (logging.Logger): Optional logger for printing information.
    """
    def __init__(self, save_dir, max_checkpoints, metric_name,
                 maximize_metric=False, log=None):
        super(CheckpointSaver, self).__init__()

        self.save_dir = save_dir
        self.max_checkpoints = max_checkpoints
        self.metric_name = metric_name
        self.maximize_metric = maximize_metric
        self.best_val = None
        self.ckpt_paths = queue.PriorityQueue()
        self.log = log
        self._print(f"Saver will {'max' if maximize_metric else 'min'}imize {metric_name}...")

    def is_best(self, metric_val):
        """Check whether `metric_val` is the best seen so far.

        Args:
            metric_val (float): Metric value to compare to prior checkpoints.
        """
        if metric_val is None:
            # No metric reported
            return False

        if self.best_val is None:
            # No checkpoint saved yet
            return True

        return ((self.maximize_metric and self.best_val < metric_val)
                or (not self.maximize_metric and self.best_val > metric_val))

    def _print(self, message):
        """Print a message if logging is enabled."""
        if self.log is not None:
            self.log.info(message)

    def save(self, step, model, metric_val, device):
        """Save model parameters to disk.

        Args:
            step (int): Total number of examples seen during training so far.
            model (torch.nn.DataParallel): Model to save.
            metric_val (float): Determines whether checkpoint is best so far.
            device (torch.device): Device where model resides.
        """
        ckpt_dict = {
            'model_name': model.__class__.__name__,
            'model_state': model.cpu().state_dict(),
            'step': step
        }
        model.to(device)

        checkpoint_path = os.path.join(self.save_dir,
                                       f'step_{step}.pth.tar')
        torch.save(ckpt_dict, checkpoint_path)
        self._print(f'Saved checkpoint: {checkpoint_path}')

        if self.is_best(metric_val):
            # Save the best model
            self.best_val = metric_val
            best_path = os.path.join(self.save_dir, 'best.pth.tar')
            shutil.copy(checkpoint_path, best_path)
            self._print(f'New best checkpoint at step {step}...')

        # Add checkpoint path to priority queue (lowest priority removed first)
        if self.maximize_metric:
            priority_order = metric_val
        else:
            priority_order = -metric_val

        self.ckpt_paths.put((priority_order, checkpoint_path))

        # Remove a checkpoint if more than max_checkpoints have been saved
        if self.ckpt_paths.qsize() > self.max_checkpoints:
            _, worst_ckpt = self.ckpt_paths.get()
            try:
                os.remove(worst_ckpt)
                self._print(f'Removed checkpoint: {worst_ckpt}')
            except OSError:
                # Avoid crashing if checkpoint has been removed or protected
                pass

def load_model(model, checkpoint_path, gpu_ids, return_step=True):
    """Load model parameters from disk.

    Args:
        model (torch.nn.DataParallel): Load parameters into this model.
        checkpoint_path (str): Path to checkpoint to load.
        gpu_ids (list): GPU IDs for DataParallel.
        return_step (bool): Also return the step at which checkpoint was saved.

    Returns:
        model (torch.nn.DataParallel): Model loaded from checkpoint.
        step (int): Step at which checkpoint was saved. Only if `return_step`.
    """
    device = f"cuda:{gpu_ids[0] if gpu_ids else 'cpu'}"
    ckpt_dict = torch.load(checkpoint_path, map_location=device)

    # Build model, load parameters
    model.load_state_dict(ckpt_dict['model_state'])

    if return_step:
        step = ckpt_dict['step']
        return model, step

    return model

def get_logger(log_dir, name):
    """Get a `logging.Logger` instance that prints to the console
    and an auxiliary file.

    Args:
        log_dir (str): Directory in which to create the log file.
        name (str): Name to identify the logs.

    Returns:
        logger (logging.Logger): Logger instance for logging events.
    """
    class StreamHandlerWithTQDM(logging.Handler):
        """Let `logging` print without breaking `tqdm` progress bars.

        See Also:
            > https://stackoverflow.com/questions/38543506
        """
        def emit(self, record):
            try:
                msg = self.format(record)
                tqdm.tqdm.write(msg)
                self.flush()
            except (KeyboardInterrupt, SystemExit):
                raise
            except:
                self.handleError(record)

    # Create logger
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)

    # Log everything (i.e., DEBUG level and above) to a file
    log_path = os.path.join(log_dir, 'log.txt')
    file_handler = logging.FileHandler(log_path)
    file_handler.setLevel(logging.DEBUG)

    # Log everything except DEBUG level (i.e., INFO level and above) to console
    console_handler = StreamHandlerWithTQDM()
    console_handler.setLevel(logging.INFO)

    # Create format for the logs
    file_formatter = logging.Formatter('[%(asctime)s] %(message)s',
                                       datefmt='%m.%d.%y %H:%M:%S')
    file_handler.setFormatter(file_formatter)
    console_formatter = logging.Formatter('[%(asctime)s] %(message)s',
                                          datefmt='%m.%d.%y %H:%M:%S')
    console_handler.setFormatter(console_formatter)

    # add the handlers to the logger
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

In [24]:
class BertFineTune(nn.Module):
    def __init__(self):
        super(BertFineTune, self).__init__()
        self.embed_model = BertModel.from_pretrained('bert-base-uncased')
        self.fc_weight = nn.Parameter(torch.zeros(768, 5), requires_grad = True)  # 5 represents the number of classes 
        self.fc_bias = nn.Parameter(torch.zeros(1,5), requires_grad = True)
        nn.init.xavier_uniform_(self.fc_weight)  # FC layer initialization
        self.softmax_layer = nn.LogSoftmax(dim = 1)  # uses logsoftmax for NLL loss instead of normal softmax

    def forward(self, x, seg_id_tensor, attnmask_tensor):
        original = torch.zeros([x.size()[0], 350, 768], dtype = torch.float32, device = 'cuda:0') #, device=device) # batchsize x max_text_len x hidden_size
        mask = torch.ones([x.size()[0], 350], dtype = torch.float32, device = 'cuda:0') #, device=device)  # batchsize x max_text_len
        sep_idxs = (x == 102).nonzero().reshape(x.size()[0], -1)  # [idx, pos of [SEP] token]

        for i in range(x.size()[0]):
            # get pre-contextual word embedding from BERT
            curr_embed = torch.zeros([1,350,768], dtype=torch.float32)
            # take the BERT embedding
            # last_hidden_states = last_layer_embedding  (batch_size, seq_len, hidden_size)
            # pooler output = last_layer_embedding further preprocessed by linear and Tanh layers
            # hidden_states = tuple of length 13 (one for output of embedding layer and 12 output for each layer in the transformer)  
            last_hidden_states, pooler_output, hidden_states = self.embed_model(x[i:i+1], token_type_ids = seg_id_tensor[i:i+1], 
                                                                                attention_mask = attnmask_tensor[i:i+1], 
                                                                                output_hidden_states = True) 
            
            # do masking and filter pad sequences
            # set embedding after the [SEP] token to 0, and also create the masking
            curr_embed = last_hidden_states
            end_text = sep_idxs[i,1].item()
            curr_embed[0,end_text+1:,:] = torch.zeros([350 - (end_text+1), 768], dtype = torch.float32)  # set everything else to 0 after [SEP] token
            mask[i,end_text+1:] = torch.zeros([350 - (end_text+1)], dtype=torch.float32)  # set everything else to 0 after [SEP] token
            original[i] = curr_embed

        # Take [CLS] token embedding which is at index 0
        cls_embed = original[:,0,:]  # size = batch_size x 768

        # attach FC layer after the CLS word embeddings
        dotted = torch.matmul(cls_embed, self.fc_weight) # size = batch_size x 5, 5 = Number of classes
        dotted = dotted.add(self.fc_bias) # add the bias  (batch_size x 5)

        # perform log softmax to normalize
        softmax_output = self.softmax_layer(dotted)

        return softmax_output

In [25]:
def get_prediction(review_ids, log_softmax_score):
    """
    review_ids (int): Tensor of Review example IDs.
    log_softmax_score (list): tensor of log likehood scores (take max to get prediction)
    """
    maxs = torch.argmax(log_softmax_score, dim = 1)

    pred_dict = {}
    for id, max_val in zip(review_ids, maxs):
        pred_dict[id.item()] = max_val.item() 
    return pred_dict

In [26]:
def evaluate_dict(gold_dict, pred_dict):
    sum_acc = 0
    total = 0
    
    for key, value in pred_dict.items():
        total += 1
        ground_truths = gold_dict[key]
        prediction = value
        if ground_truths == prediction:
            sum_acc += 1

    eval_dict = {'acc': 100. * sum_acc / total}

    return eval_dict

In [27]:
def evaluate(model, data_loader, eval_file, device):
    nll_meter = AverageMeter()

    model.eval()
    pred_dict = {}

    # get all true labels for ratings
    test_dataset = np.load(eval_file)
    true_labels = torch.from_numpy(test_dataset['rating']).long()
    uniq_ids = torch.from_numpy(test_dataset['ids']).long()
    true_labels = true_labels - 1 ## Need to subtract 1 because classes need to be 0 to n_classes - 1
    gold_dict = {}
    for true, ids in zip(true_labels, uniq_ids):
        gold_dict[ids.item()] = true.item()

    with torch.no_grad(), tqdm.notebook.tqdm(total=len(data_loader.dataset), position=1, leave=True) as progress_bar:
        for text, attnmask, seg_id, rating, ids  in data_loader:
            # Setup for forward
            text = text.to(device)
            attnmask = attnmask.to(device)
            seg_id = seg_id.to(device)
            batch_size = text.size(0)
            ids = ids.to(device)

            # Forward
            log_softmax_scores = model(text, seg_id, attnmask)
            rating = rating.to(device)
            # rating needs to be 0 to num_classes - 1
            rating = rating - 1

            loss = F.nll_loss(log_softmax_scores, rating)
            nll_meter.update(loss.item(), batch_size)

            # Get maximum prediction for prediction
            preds = get_prediction(ids, log_softmax_score)
            pred_dict.update(preds)

            # Log info
            progress_bar.update(batch_size)
            progress_bar.set_postfix(NLL=nll_meter.avg)

    model.train()

    results = evaluate_dict(gold_dict, pred_dict)
    results_list = [('NLL', nll_meter.avg),
                    ('acc', results['acc'])]

    results = OrderedDict(results_list)
    return results, pred_dict

In [28]:
def visualize(tbx, pred_dict, eval_path, step, split, num_visuals):
    """Visualize text examples to TensorBoard.

    Args:
        tbx (tensorboardX.SummaryWriter): Summary writer.
        pred_dict (dict): dict of predictions of the form id -> pred.
        eval_path (str): Path to eval JSON file.
        step (int): Number of examples seen so far during training.
        split (str): Name of data split being visualized.
        num_visuals (int): Number of visuals to select at random from preds.
    """
    if num_visuals <= 0:
        return
    if num_visuals > len(pred_dict):
        num_visuals = len(pred_dict)

    visual_ids = np.random.choice(list(pred_dict), size=num_visuals, replace=False)

    with open(eval_path, 'r') as eval_file:
        eval_dict = json.load(eval_file)
    for i, id_ in enumerate(visual_ids):
        pred = pred_dict[id_] + 1 # NEED POST PROCESSING BECAUSE WE SUBTRACTED TO COMPLY WITH NLL LOSS LABELS (0, n_classes - 1)
        example = eval_dict[str(id_)]
        user_id = example['user_id']
        business_id = example['business_id']
        text = example['text']
        gold = int(example['rating'])

        tbl_fmt = (f'- **Reviews:** {text}\n'
                   + f'- **Answer:** {gold}\n'
                   + f'- **Prediction:** {pred}')
        tbx.add_text(tag=f'{split}/{i+1}_of_{num_visuals}',
                     text_string=tbl_fmt,
                     global_step=step)

In [29]:
save_dir = 'gdrive/My Drive/Yelp_Sentiment_Analysis/save/train/baseline-01'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

log = get_logger(save_dir, 'baseline')
tbx = SummaryWriter(save_dir)
log.info(f'Using random seed 224 ...')

saver = CheckpointSaver(save_dir,
                        max_checkpoints=5,
                        metric_name='acc',
                        maximize_metric='acc',
                        log=log)

model = BertFineTune()
model = model.to('cuda:0')
model.train()
optimizer = optim.Adam(model.parameters(), lr = 0.00005)
#scheduler = sched.LambdaLR(optimizer, lambda s: 1.)  # Constant LR

# LEARNING RATE SCHEDULER
# step_size: at how many multiples of epoch you decay
# step_size = 1, after every 1 epoch, new_lr = lr*gamma 
# step_size = 2, after every 2 epoch, new_lr = lr*gamma 
# gamma = decaying factor
#scheduler = StepLR(optimizer, step_size=1, gamma=0.9)


ema = EMA(model, 0.999)

[08.01.20 14:18:23] Using random seed 224 ...
[08.01.20 14:18:23] Saver will maximize acc...


In [ ]:
epoch = 0
step = 0
steps_till_eval = 30000 # evaluate model after 30000 iterations
device = 'cuda:0'
while epoch != 3:  # Num Epochs to train on 
    epoch += 1
    #scheduler.step()     # Decay Learning Rate
    log.info(f'Starting epoch {epoch}...')
    with torch.enable_grad(), tqdm.tqdm(total=len(train_loader.dataset), position=0, leave=True) as progress_bar:
        for text, attnmask, seg_id, rating, ids in train_loader:
            # Setup for forward
            text = text.to(device)
            attnmask = attnmask.to(device)
            seg_id = seg_id.to(device)
            batch_size = text.size(0)
            optimizer.zero_grad()

            # Forward
            log_softmax_score = model(text, seg_id, attnmask)
            rating = rating.to(device)
            
            # need ratings to be 0 to n_classes - 1 (can later transform it)
            rating = rating - 1
            
            loss = F.nll_loss(log_softmax_score, rating)  #NLL loss for correct position
            loss_val = loss.item()

            # Backward
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 3)  # clip max gradients to 3
            optimizer.step()
            ema(model, step // batch_size)

            # Log info
            step += batch_size
            progress_bar.update(batch_size)
            progress_bar.set_postfix(epoch=epoch, NLL=loss_val)
            
            tbx.add_scalar('train/NLL', loss_val, step)
            tbx.add_scalar('train/LR',
                             optimizer.param_groups[0]['lr'],
                             step)
            
            steps_till_eval -= batch_size
            if steps_till_eval <= 0:
                steps_till_eval = 30000

                # Evaluate and save checkpoint
                log.info(f'Evaluating at step {step}...')
                ema.assign(model)
                results, pred_dict = evaluate(model, dev_loader, 'gdrive/My Drive/Yelp_Sentiment_Analysis/test.npz', device)
                
                saver.save(step, model, results['acc'], device)
                ema.resume(model)

                # Log to console
                results_str = ', '.join(f'{k}: {v:05.2f}' for k, v in results.items())
                log.info(f'Dev {results_str}')

                # Log to TensorBoard
                log.info('Visualizing in TensorBoard...')
                for k, v in results.items():
                     tbx.add_scalar(f'dev/{k}', v, step)

                visualize(tbx,
                          pred_dict=pred_dict,
                          eval_path='dataset/test.json',
                          step=step,
                          split='dev',
                          num_visuals=30)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  0%|          | 0/90000 [00:00<?, ?it/s]

[08.01.20 14:18:50] Starting epoch 1...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  del sys.path[0]
 33%|███▎      | 30000/90000 [29:24<59:10, 16.90it/s, NLL=0.759, epoch=1]

[08.01.20 14:48:14] Evaluating at step 30000...


 33%|███▎      | 30000/90000 [29:34<59:10, 16.90it/s, NLL=0.759, epoch=1]

...

 67%|██████▋   | 60000/90000 [1:05:00<29:34, 16.90it/s, NLL=1.06, epoch=1]

[08.01.20 15:23:51] Saved checkpoint: gdrive/My Drive/Yelp_Sentiment_Analysis/save/train/baseline-01/step_60000.pth.tar


 67%|██████▋   | 60000/90000 [1:05:03<29:34, 16.90it/s, NLL=1.06, epoch=1]

[08.01.20 15:23:54] New best checkpoint at step 60000...
[08.01.20 15:23:54] Dev NLL: 00.81, acc: 20.19
[08.01.20 15:23:54] Visualizing in TensorBoard...


 99%|█████████▉| 89064/90000 [1:33:37<00:54, 17.07it/s, NLL=1.13, epoch=1] 

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs, output_hidden_states = True)

last_hidden_states, pooler_output, hidden_states = outputs  # The last hidden-state is the first element of the output tuple

In [ ]:
inputs['input_ids'].size()

In [ ]:
last_hidden_states.size()

In [ ]:
hidden_states

In [ ]:
hidden_states[1]

In [ ]:
len(hidden_states)

In [ ]:
last_hidden_states.size()

In [ ]:
hidden_states[].size()

In [ ]:
model.config